In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
from utility_functions import FDRthreshold_js, ROC_PR
from sklearn.metrics import confusion_matrix
import os

## SET SEED FOR REPRODUCIBILITY
random.seed(100)

plt.style.use('seaborn')

results_path = os.path.expandvars('$SCRATCH/RESULTS/modifiedMainQSO_minsignal1/A/')
plot_path = os.path.expandvars('$SCRATCH/PLOTS/modifiedMainQSO_minsignal1/A/')

with open(os.path.join(results_path, 'results.pkl'), "rb") as f:
    results = pickle.load(f)

# Hyperparameters: results['hyperparameters']
# Probabilities results['results']['CNN1']['Y_pred_prob']

y_test = np.array(results['y_test'])[:, 1]
y_pred_prob = np.array(results['results']['Y_pred_prob'])[:, 1]

In [ ]:
# =============================================================================
# # =============================================================================
# # ROC & PR curves
# # =============================================================================
# =============================================================================


ls_roc_auc, ls_fpr, ls_tpr, ls_pr_auc, ls_precision, ls_recall, pr_no_skill, ls_roc_auc_noskill, ls_fpr_noskill, ls_tpr_noskill = ROC_PR(y_pred_prob, y_test)

#ROC
plt.figure()
#plt.plot(ls_fpr, ls_tpr, lw=1, color=color_ls[met], label=met+" (AUC={})".format(np.round(ls_roc_auc[met],3)))
plt.plot(ls_fpr, ls_tpr, lw=1, label=f"ROC-AUC={ls_roc_auc:.3f})")
plt.plot(ls_fpr_noskill, ls_tpr_noskill, linestyle='--', lw=1, color='gray')  #'No Skill'
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.title('ROC-Curve')
plt.legend()
# plt.savefig(visualisation_path + 'ROC_' + planet + '_alpha' + str(alpha) + '_bal' + str(bal) + '_combined_CV_' + str(plotname) + '_version' + str(version) + 'frame' + str(frame) + '_fold' + str(j) + '_js.pdf')
plt.show()


#PR
plt.figure()
#plt.plot(ls_recall, ls_precision[met], lw=1, color=color_ls[met], label=met + " (AUC={})".format(np.round(ls_pr_auc[met],3)))
plt.plot(ls_recall, ls_precision, lw=1, label=f"PR-AUC={ls_pr_auc:.3f})")
plt.plot([0, 1], [pr_no_skill, pr_no_skill], linestyle='--', lw=1, color='gray')
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.title('PR-Curve')
plt.legend()
#plt.savefig(visualisation_path + 'PR_' + planet + '_alpha' + str(alpha) + '_bal' + str(bal) + '_combined_CV_' + str(plotname) + '_version' + str(version) + 'frame' + str(frame) + '_fold' + str(j) + '_js.pdf')
plt.show()


In [ ]:

# =============================================================================
# # =============================================================================
# # Histogram
# # =============================================================================
# =============================================================================

prob_treshold = FDRthreshold_js(y_test, y_pred_prob)
CM = confusion_matrix(y_test, y_pred_prob>=prob_treshold)  # [['tn', 'fp'], ['fn', 'tp']]
    
# Plot Histograms
plt.figure()
x1 = np.array(y_pred_prob[y_test == 1])
x1_med = np.median(x1)
weightsx1 = np.ones_like(x1) / len(x1)
x2 = np.array(y_pred_prob[y_test == 0])
x2_med = np.median(x2)
weightsx2 = np.ones_like(x2) / len(x2)
kwargs = dict(histtype='stepfilled', bins=60, density=False)
plt.hist(x1, **kwargs, color='deepskyblue', alpha=0.5, edgecolor='xkcd:azure', weights=weightsx1)
plt.axvline(x1_med, color="steelblue", alpha=0.7, lw=2, label='H2O group,\n median = ' + str(np.round(x1_med, 2)))
plt.hist(x2, **kwargs, color='crimson', alpha=0.5, edgecolor='red', weights=weightsx2)
plt.axvline(x2_med, color="indianred", alpha=0.7, lw=2, label=' No H2O group,\n median = ' + str(np.round(x2_med, 2)))
plt.axvline(prob_treshold, color='rebeccapurple', alpha=0.7, label='FDR=0.05: T*=' + str(np.round(prob_treshold,3)), lw=2) # Threshold for 5% False Discovery Rate (FDR)
plt.ylabel('Relative Frequency', fontsize=20, color='black')
plt.xlabel('Probability scores', fontsize=20, color='black')
plt.xlim([0, 1])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title('Empirical distribution of scores', fontsize=22, color="black")
plt.legend(fontsize=12)
#plt.savefig(visualisation_path + 'Prob_signals_method' + str(met) + 'alpha' + str(alpha) + '_bal' + str(bal) + '_version' + str(version) + 'frame' + str(frame) + '_js.pdf')
plt.show()

In [ ]:
# =============================================================================
# # =============================================================================
# # Classification
# # =============================================================================
# =============================================================================

# if baseline_included:
#     X=np.arange(len(methods_ls))
#     fp=[CM[met][0,1] for met in methods_ls]
#     tp=[CM[met][1,1] for met in methods_ls]
#     methods_ls1 = methods_ls
# else:
#     X = np.arange(len(methods_ls)+2)
#     fp = [CM[met][0, 1] for met in methods_ls] + [CNN1_fp, PCT_fp]
#     tp = [CM[met][1, 1] for met in methods_ls] + [CNN1_tp, PCT_tp]
#     methods_ls1 = methods_ls+['CNN1_bl','PCT_bl']
# 
# metlen=max([len(met) for met in methods_ls])
# size=min(14, np.round(14*6*6/len(methods_ls)/metlen))

# plt.figure(figsize=(16.0, 5.5))
# plt.bar(X - 0.2, fp, 0.4, label='False Positives', color="crimson", alpha=0.5, edgecolor='red')
# plt.bar(X + 0.2, tp, 0.4, label='True Positives', color="deepskyblue", alpha=0.5, edgecolor='xkcd:azure')
# for i,x in enumerate(X):
#     plt.text(x - 0.2, fp[i] + 15, str(fp[i]), ha='center', fontsize=12)
#     plt.text(x + 0.2, tp[i] + 15, str(tp[i]), ha='center', fontsize=12)
# plt.tick_params(labelsize=16)
# plt.xticks(X, methods_ls1, size=16)
# plt.xlabel("Method", fontsize=20, color="black")
# plt.ylabel("Planet Detections", fontsize=20, color="black")
# plt.ylim(0, max(*fp,*tp)*1.25)
# plt.xlim(-0.6, 5.6)
# plt.axvline(x=2.5, color='grey', linestyle=':', linewidth=2)
# plt.title("Classification results for a 5% FDR threshold (v06)", fontsize=22, color="black")
# plt.legend(framealpha=0.1, facecolor='gray', prop=dict(size=12), loc='upper left', fontsize=12)
# plt.tight_layout()
# if SAVE: plt.savefig(visualisation_path + 'FDR_results_alpha' + str(alpha) + '_version' + str(version) + '_js.pdf', bbox_inches='tight')
# plt.show()

In [ ]:
# # Saving Baseline model Results
# baseline_path = visualisation_path + 'BaselineForPlotting/'
# np.savez(baseline_path+'baselineplotting_rocpr_GQlupb_data_0_alpha_'+str(alpha)+'_nfolds_'+str(len(folds))+'_CNN1.npz', array1=ls_fpr['CNN1'], array2=ls_tpr['CNN1'], array3=ls_recall['CNN1'], array4=ls_precision['CNN1'])
# np.savez(baseline_path+'baselineplotting_rocpr_GQlupb_data_0_alpha_'+str(alpha)+'_nfolds_'+str(len(folds))+'_PCT.npz', array1=ls_fpr['PCT'], array2=ls_tpr['PCT'], array3=ls_recall['PCT'], array4=ls_precision['PCT'])
# with open(baseline_path+'baselineplotting_class_GQlupb_data_0_alpha_'+str(alpha)+'_nfolds_'+str(len(folds))+'_CNN1.pkl', "wb") as f:
#     pickle.dump((CM['CNN1'][0,1], CM['CNN1'][1,1], ls_roc_auc['CNN1'], ls_pr_auc['CNN1']), f)
# with open(baseline_path+'baselineplotting_class_GQlupb_data_0_alpha_'+str(alpha)+'_nfolds_'+str(len(folds))+'_PCT.pkl', "wb") as f:
#     pickle.dump((CM['PCT'][0,1], CM['PCT'][1,1], ls_roc_auc['PCT'], ls_pr_auc['PCT']), f)
